In [ ]:
import MetaTrader5 as mt5
import pandas as pd
from datetime import datetime
import time
import requests
import pandas_ta as pta

In [ ]:
# function to send a market order
def market_order(symbol, volume, order_type, **kwargs):
    tick = mt5.symbol_info_tick(symbol)

    order_dict = {'buy' : 0, 'sell' : 1}
    price_dict = {'buy' : tick.ask, 'sell' : tick.bid}

    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": volume,
        "type": order_dict[order_type],
        "price": price_dict[order_type],
        "deviation": DEVIATION,
        "magic": 22085,
        "comment": "python market order",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC,
    }

    order_result = mt5.order_send(request)
    print(order_result)
    return order_result

In [ ]:
# function to close an order base on the ticket id
def close_order(ticketID):
    positionIDs = mt5.positions_get()

    for pos in positionIDs:
        tick = mt5.symbol_info_tick(pos.symbol) #current price
        type_dict = {0:1, 1:0} # 0 represents buy, 1 represents sell - inverting order_type to close the position
        price_dict = {0 : tick.ask, 1 : tick.bid}

        if pos.ticket == ticketID:
            request = {
                "action": mt5.TRADE_ACTION_DEAL,
                "position": pos.ticket,
                "symbol": pos.symbol,
                "volume": pos.volume,
                "type": type_dict[pos.type],
                "price": price_dict[pos.type],
                "deviation": DEVIATION,
                "magic": 22085,
                "comment": "python close order",
                "type_time": mt5.ORDER_TIME_GTC,
                "type_filling": mt5.ORDER_FILLING_IOC,
            }

            order_result = mt5.order_send(request)
            print(order_result)
            return order_result
    return "TicketID does not exist."

In [ ]:
# function to get the exposure of a symbol
def get_exposure(symbol):
    positions = mt5.positions_get(symbol = symbol)
    if positions :
        pos_df = pd.DataFrame(positions, columns=positions[0]._asdict().keys())
        exposure = pos_df['volume'].sum()
        return exposure

In [ ]:
# function to look for trading signals, what trend is now?
def signal(symbol, timeframe, rsi_period, overbought, oversold) :
    bars = mt5.copy_rates_from_pos(symbol, timeframe, 1, 100)
    bars_df = pd.DataFrame(bars)
    
    last_close = bars_df.iloc[-1].close
    
    bars_df['rsi'] = pta.rsi(bars_df.close, rsi_period)
    rsi = bars_df.iloc[-1].rsi

    status = 'normal'
    if rsi > overbought:
        status = 'sell'
    elif rsi < oversold:
        status = 'buy'
    
    return last_close, rsi, status

In [ ]:
if __name__ == '__main__' :

    #PARAMETERS -----------
    SYMBOL = "XAUUSD"
    VOLUME = 1.0
    TIMEFRAME = mt5.TIMEFRAME_H1
    RSI_PERIOD = 14
    OVERBOUGHT = 80
    OVERSOLD = 20
    DEVIATION = 100

    #Connect and login
    mt5.initialize()
    login = 121391564
    password = 'xxxx'
    server = 'Exness-MT5Trial7'

    mt5.login(login, password, server)

    print('Account :', mt5.account_info().login)
    print('Balance :', mt5.account_info().balance)
    print('Equity :', mt5.account_info().equity)
    print('Exposure :', get_exposure(SYMBOL))

    #chatbot
    token = "5868960988:AAHXC_mJbjt4EA_aknIBBp5NK6ddLb9HDU0"
    url = f"https://api.telegram.org/bot{token}"
    
    while True :
        #Run every hour starts
        while datetime.now().minute == 0:
            exposure = get_exposure(SYMBOL)

            # calculating last candle close and simple moving average and checking for trading signal
            last_close, rsi, status = signal(SYMBOL, TIMEFRAME, RSI_PERIOD, OVERBOUGHT, OVERSOLD)

            if status == 'buy':
                # if we have a BUY signal, close all short positions
                for pos in mt5.positions_get():
                    if pos.type == 1:
                        close_order(pos.ticket)
                        params = {"chat_id": "5297105149", "text": f"Close [Sell BTCUSD], Balance : {mt5.account_info().balance}, RSI : {round(rsi,2)}"}
                        r = requests.get(url + "/sendMessage", params=params)
                
                # if there are no open positions, open a new long position
                if not mt5.positions_total():
                    market_order(SYMBOL, VOLUME, status)
                    params = {"chat_id": "5297105149", "text": f"Open [Buy BTCUSD], Balance : {mt5.account_info().balance}, RSI : {round(rsi,2)}"}
                    r = requests.get(url + "/sendMessage", params=params)
            
            elif status == 'sell':
                # if we have a SELL signal, close all short positions
                for pos in mt5.positions_get():
                    if pos.type == 0:
                        close_order(pos.ticket)
                        params = {"chat_id": "5297105149", "text": f"Close [Buy BTCUSD], Balance : {mt5.account_info().balance}, RSI : {round(rsi,2)}"}
                        r = requests.get(url + "/sendMessage", params=params)

                # if there are no open positions, open a new short position
                if not mt5.positions_total():
                    market_order(SYMBOL, VOLUME, status)
                    params = {"chat_id": "5297105149", "text": f"Open [Sell BTCUSD], Balance : {mt5.account_info().balance}, RSI : {round(rsi,2)}"}
                    r = requests.get(url + "/sendMessage", params=params)

            else :
                for pos in mt5.positions_get():
                    if (pos.type == 0) & (rsi > 50):
                        close_order(pos.ticket)
                        params = {"chat_id": "5297105149", "text": f"Close [Buy BTCUSD], Balance : {mt5.account_info().balance}, RSI : {round(rsi,2)}"}
                        r = requests.get(url + "/sendMessage", params=params)

                    elif (pos.type == 1) & (rsi < 50):
                        close_order(pos.ticket)
                        params = {"chat_id": "5297105149", "text": f"Close [Buy BTCUSD], Balance : {mt5.account_info().balance}, RSI : {round(rsi,2)}"}
                        r = requests.get(url + "/sendMessage", params=params)

            #print('time: ', datetime.now())
            print(f'exposure: {exposure}, Equity : {mt5.account_info().equity}, Price : {last_close}, RSI : {round(rsi,2)}')
            #print('RSI: ', round(rsi,2))
            #print('signal: ', status)
            #print('-------\n')

            # update every 60 s > once per hour
            time.sleep(60)